In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [3]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: shuffled_binary_fullrank
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 3.024179620034542,
           'expected_sample_data_size': 16.515151515151516,
           'expected_time_to_recover': 13.992099999999999,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3323790471590991,
             'bit_redundancy': 46.707095671225844,
             'ideal_bit_efficency': 0.39324279034806525,
             'ideal_bit_redundancy': 35.87903216978748,
             'ideal_packet_efficiency': 1.9662139517403265,
             'ideal_packet_redundancy': 7.175806433957494,
             'ideal_time_to_recover': 11.826487299712324,
             'ideal_transmitted_bits': 59.13243649856163,
             'packet_efficiency': 1.6618952357954957,
             'packet_redundancy': 9.341419134245168,
             'permeability': 0.8452260418173345,
             'saturation': 4.650680865754831,
             'transmitted_bits': 69.9605}}
Found implementation: single_binary_fullrank
  No 

{'basic': {'expected_sample_burst_size': 2.965405062286757,
           'expected_sample_data_size': 16.523129413631676,
           'expected_time_to_recover': 13.2806,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3501860507623775,
             'bit_redundancy': 43.14959567122584,
             'ideal_bit_efficency': 0.41303392065899885,
             'ideal_bit_redundancy': 33.045614143299915,
             'ideal_packet_efficiency': 2.0651696032949944,
             'ideal_packet_redundancy': 6.609122828659983,
             'ideal_time_to_recover': 11.259803694414813,
             'ideal_transmitted_bits': 56.29901847207407,
             'packet_efficiency': 1.7509302538118874,
             'packet_redundancy': 8.629919134245169,
             'permeability': 0.8478384782626398,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.40299999999999}}
Found implementation: single_binary_fullrank2
  Fil

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3480.20it/s]


{'basic': {'expected_sample_burst_size': 2.988347374222502,
           'expected_sample_data_size': 16.533894968900086,
           'expected_time_to_recover': 13.855899999999998,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.33564624930569875,
             'bit_redundancy': 46.02609567122583,
             'ideal_bit_efficency': 0.3963111797208847,
             'ideal_bit_redundancy': 35.42120673102773,
             'ideal_packet_efficiency': 1.981555898604423,
             'ideal_packet_redundancy': 7.084241346205548,
             'ideal_time_to_recover': 11.734922211960379,
             'ideal_transmitted_bits': 58.67461105980188,
             'packet_efficiency': 1.6782312465284936,
             'packet_redundancy': 9.205219134245167,
             'permeability': 0.8469260179389559,
             'saturation': 4.650680865754831,
             'transmitted_bits': 69.27949999999998}}
